In [19]:
import pandas as pd
import glob
import os
from datetime import datetime, timedelta, timezone

def process_climate_files(input_directory, output_file):
    # Use glob to find all files matching the pattern in the input directory
    file_pattern = os.path.join(input_directory, 'hourly_weather_data_*.csv')
    climate_files = glob.glob(file_pattern)
    
    # Debugging: Print the list of files found
    print("Files found:", climate_files)

    # Check if files are found before proceeding
    if not climate_files:
        print("Directory contents:")
        print(os.listdir(input_directory))
        raise ValueError("No climate temperature files found. Please check the directory and file pattern.")
    
    all_data = []  # List to hold all DataFrames
    
    for file in climate_files:
        # Extract the date from the filename
        filename = os.path.basename(file)
        date_str = filename.split('_')[-1].split('.')[0]  # Extract date part
        date = datetime.strptime(date_str, '%d%m%Y').date()  # Convert to datetime.date

        # Load the climate temperature dataset
        df = pd.read_csv(file)
        
        # Debugging: Print the column names
        print(f"Processing file: {file}")
        print("Column names:", df.columns)
        
        # Check if 'Time' and 'Temperature' columns exist
        if 'Time' not in df.columns or 'Temperature' not in df.columns:
            print(f"Required columns not found in {file}. Skipping this file.")
            continue
        
        # Convert the Time column to time format and then combine with the extracted date
        df['Time'] = pd.to_datetime(df['Time']).dt.time  # Convert to time only
        df['Time'] = df['Time'].apply(lambda t: datetime.combine(date, t).replace(tzinfo=timezone(timedelta(hours=8))).strftime('%Y-%m-%d %H:%M:%S%z'))
        
        # Change temperature values to integers by stripping non-numeric characters
        df['Temperature'] = df['Temperature'].str.extract('(\d+)').astype(int)
        
        # Change humidity values to floats by stripping the percentage sign
        if 'Humidity' in df.columns:
            df['Humidity'] = df['Humidity'].str.rstrip('%').astype('float') / 100.0
        
        # Append the DataFrame to the list
        all_data.append(df)
    
    # Concatenate all DataFrames into one
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Ensure the combined DataFrame is ordered chronologically by the 'Time' column
    combined_df['Time'] = pd.to_datetime(combined_df['Time'])
    combined_df = combined_df.sort_values(by='Time').reset_index(drop=True)
    
    # Save the combined DataFrame to the output file
    combined_df.to_csv(output_file, index=False)
    print(f"Processed and saved combined file: {output_file}")

# Define the input directory and output file path
input_directory = 'training/'
output_file = 'training/clean/combined_climate_data.csv'

# Process all climate files and combine them into one
process_climate_files(input_directory, output_file)


Files found: ['training\\hourly_weather_data_15032023.csv', 'training\\hourly_weather_data_15042023.csv', 'training\\hourly_weather_data_15052023.csv', 'training\\hourly_weather_data_16032023.csv', 'training\\hourly_weather_data_16042023.csv', 'training\\hourly_weather_data_16052023.csv', 'training\\hourly_weather_data_17032023.csv', 'training\\hourly_weather_data_17042023.csv', 'training\\hourly_weather_data_17052023.csv', 'training\\hourly_weather_data_18032023.csv', 'training\\hourly_weather_data_18042023.csv', 'training\\hourly_weather_data_18052023.csv', 'training\\hourly_weather_data_19032023.csv', 'training\\hourly_weather_data_19042023.csv', 'training\\hourly_weather_data_19052023.csv', 'training\\hourly_weather_data_20032023.csv', 'training\\hourly_weather_data_20042023.csv', 'training\\hourly_weather_data_20052023.csv', 'training\\hourly_weather_data_21032023.csv', 'training\\hourly_weather_data_21042023.csv', 'training\\hourly_weather_data_21052023.csv', 'training\\hourly_we

C:\Users\Vincent\AppData\Local\Temp\ipykernel_19856\2381872865.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time']).dt.time  # Convert to time only
C:\Users\Vincent\AppData\Local\Temp\ipykernel_19856\2381872865.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time']).dt.time  # Convert to time only
C:\Users\Vincent\AppData\Local\Temp\ipykernel_19856\2381872865.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time']).dt.time  # Convert to time only
C:\Users\Vincent

In [18]:
import pandas as pd
import os

def combine_and_resample_device_files(device_files, output_file):
    all_device_data = []  # List to hold all DataFrames
    
    for file in device_files:
        # Load the device temperature dataset
        df = pd.read_csv(file)
        
        # Convert the time column to datetime format
        df['time'] = pd.to_datetime(df['time'])
        
        # Set the time column as index for resampling
        df.set_index('time', inplace=True)
        
        # Resample to hourly frequency and reset index
        df_resampled = df.resample('H').mean().reset_index()
        
        # Append the resampled DataFrame to the list
        all_device_data.append(df_resampled)
    
    # Concatenate all DataFrames into one
    combined_device_df = pd.concat(all_device_data, ignore_index=True)
    
    # Save the combined DataFrame to the output file
    combined_device_df.to_csv(output_file, index=False)
    print(f"Processed and saved combined device file: {output_file}")

# Define the input files and output file path for device data
device_files = [
    'training/march_device_clean.csv',
    'training/april_device_clean.csv',
    'training/may_device_clean.csv'
]
device_output_file = 'training/clean/combined_device_data.csv'

# Combine and resample all device temperature files into one
combine_and_resample_device_files(device_files, device_output_file)


Processed and saved combined device file: training/clean/combined_device_data.csv
